# Experimenting with EEG data

In [1]:
import sklearn

In [2]:
import os

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [57]:
RECORDINGS_FOLDER = os.path.join('..', '..', 'recordings')

# TODO(andrei): Trim start/end!
RECORDINGS_TRAIN = {
    # 'left-02.csv', 
#     'left': ['left-01.csv', 'left-02.csv'],
#     'right': ['right-01.csv', 'right-02.csv'], #, 'right-03.csv'],
#     'lr': ['left-01.csv', 'left-02.csv', 'right-01.csv', 'right-03.csv'],
      'relaxed': ['closed-relax-01.csv', 'closed-relax-02.csv', 'closed-relaxed-04.csv', 'closed-relaxed-05.csv'],
#     'forward': ['forward-01.csv'],  # TODO(andrei): Maybe 'run-01-andrei.csv'.
#     'helicopter': ['helicopter-andrei.csv'],
#       'baseline': ['baseline-andrei.csv', 'internet-browsing-01.csv'],
      'tense': ['typingtest-01.csv', 'typingtest-03.csv', 'typingtest-04.csv',
                'typingtest-05-duolingo.csv', #'typingtest-06-duolingo.csv',
                'typingtest-07.csv']
}

RECORDINGS_VALID = {
#     'lr': ['right-03.csv', 'left-03.csv'],
#     'right': [],
#     'forward': [],
#     'helicopter': [],
#     'baseline': ['internet-browsing-01.csv']
    'relaxed': ['closed-relaxed-03.csv'],
    'tense': ['typingtest-02.csv'],
#     'baseline': []
}


MUSE_LABELS = {
 '/muse/acc',
 '/muse/batt',
 '/muse/config',
 '/muse/drlref',
 '/muse/eeg',
 '/muse/eeg/quantization',
 '/muse/elements/alpha_absolute',
 '/muse/elements/alpha_relative',
 '/muse/elements/alpha_session_score',
 '/muse/elements/beta_absolute',
 '/muse/elements/beta_relative',
 '/muse/elements/beta_session_score',
 '/muse/elements/blink',
 '/muse/elements/delta_absolute',
 '/muse/elements/delta_relative',
 '/muse/elements/delta_session_score',
 '/muse/elements/experimental/concentration',
 '/muse/elements/experimental/mellow',
 '/muse/elements/gamma_absolute',
 '/muse/elements/gamma_relative',
 '/muse/elements/gamma_session_score',
 '/muse/elements/horseshoe',
 '/muse/elements/is_good',
 '/muse/elements/jaw_clench',
 '/muse/elements/low_freqs_absolute',
 '/muse/elements/raw_fft0',
 '/muse/elements/raw_fft1',
 '/muse/elements/raw_fft2',
 '/muse/elements/raw_fft3',
 '/muse/elements/theta_absolute',
 '/muse/elements/theta_relative',
 '/muse/elements/theta_session_score',
 '/muse/elements/touching_forehead',
 '/muse/version'}

# 4 electrodes, 4 sets of FFT coefficients per window!
RAW_FFT0 = '/muse/elements/raw_fft0'
RAW_FFT1 = '/muse/elements/raw_fft1'
RAW_FFT2 = '/muse/elements/raw_fft2'
RAW_FFT3 = '/muse/elements/raw_fft3'
IS_GOOD = '/muse/elements/is_good'
# interesting_feats = ['/muse/elements/raw_fft0', 'alpha_absolute']

In [58]:
def read_rec(fpath: str):
    all_ts = []
    all_fft = []
    all_good_masks = []
    
    # TODO(andrei): the IS_GOOD may contain too little info, maybe. Should try horseshoe and a thresh of like <1.25 or so.
    
    last_good = [0, 0, 0, 0]
    last_good_time = -1
    
    current_feat = None
    
    # We expect fft indices always in 0-1-2-3 order. This variable keeps track of this.
    expecting = 0
    
    with open(fpath, 'r') as f:
        for line_idx, line in enumerate(f.readlines()):            
            parts = line.split(', ')
            ts = float(parts[0])
            label = parts[1]
            
            if label.startswith('/muse/elements/raw_fft'):
                rest_np = np.array([float(part) for part in parts[2:]])
                idx = int(label[-1])
                if idx != expecting:
                    print("WRONG")
                    raise ValueError();
                else:
                    if current_feat is None:
                        current_feat = rest_np
                    else:
                        current_feat = np.hstack((current_feat, rest_np))
                    
                    expecting = (idx + 1) % 4
                    
                    if expecting == 0:
                        if last_good_time != -1 and (ts - last_good_time) > 0.005 and (ts - last_good_time) > 0.00:
                            print("WARNING: Bad data sync.")
                  
                        all_ts.append(ts)
                        all_fft.append(current_feat)
                        all_good_masks.append(np.all(last_good))
#                         print('cfs', current_feat.shape)
                        current_feat = None
  
            
#             if label == RAW_FFT0:

#                 all_fft.append(rest_np)
#                 all_fft1.append(last_fft1)
#                 all_good_masks.append(np.all(last_good))
                  
#                 # TODO(andrei): WARNING, this tolerance is HUGE, so may be problematic.
#                 if last_fft1_time != -1 and (ts - last_fft1_time) > 0.18 and (ts - last_fft1_time) > 0.00:
#                     print("WARNING: Bad data sync (fft1).", ts - last_fft1_time)
#             elif label == RAW_FFT1:
#                 last_fft1_time = ts
#                 rest_np = np.array([float(part) for part in parts[2:]])
#                 if(rest_np.shape != (129,)):
#                     print("WTF:", rest_np.shape)
#                 last_fft1 = rest_np
            elif label == IS_GOOD:
                last_good_time = ts
                rest_np = np.array([float(part) for part in parts[2:]])
                last_good = rest_np
                
#     print(len(all_fft))
#     print(len(all_fft1))
#     print(len(all_fft[0]))
#     print(len(all_fft1[0]))
#     print(np.array(all_fft).shape)
#     print(np.array(all_fft1).shape)

    print(len(all_fft))
    print(np.array(all_fft).shape)
    return np.array(all_good_masks), np.array(all_ts), np.array(all_fft)

In [59]:
def spec2data(recording_map):
    data_map = {}
    for label, fnames in recording_map.items():
        cfeats = []
        for fname in fnames:
            # Read the data for that recording for that 
            good_mask, all_ts, all_feats = read_rec(os.path.join(RECORDINGS_FOLDER, fname))
    #         all_ts = all_ts[good_mask]
            print(all_feats.shape, good_mask.shape)
            # Remove some edge frames, since they can be noisy due to the nature of the experiments.
            EDGE_CUTOFF = 10
            all_feats = all_feats[good_mask, :]
            print(all_feats.shape)
            all_feats = all_feats[EDGE_CUTOFF:, :]
            print(all_feats.shape)
            
            # Clump up all k consecutive entries into a single one (poor man's CNNs).
            FACTOR = 2
            # Need to shave off a little at the end.
            shave_right = all_feats.shape[0] % FACTOR
            if shave_right > 0:
                all_feats = all_feats[:-shave_right, :]
            all_feats = all_feats.reshape(-1, all_feats.shape[1] * FACTOR) #, order='C')
           
            print("Appending to feats the following shape:", all_feats.shape)
            cfeats.append(all_feats)

        if len(cfeats) > 1:
            data_map[label] = np.vstack(cfeats)
        elif len(cfeats) == 1:
            data_map[label] = cfeats[0]
        else:
            data_map[label] = np.array([])
    
    for label, data in data_map.items():
        print("{0}: {1}".format(label, data.shape))
        
    return data_map

print("Processing train...")
data_map_train = spec2data(RECORDINGS_TRAIN)
print("Processing validation...")
data_map_test = spec2data(RECORDINGS_VALID)

Processing train...
1235
(1235, 516)
(1235, 516) (1235,)
(1230, 516)
(1220, 516)
Appending to feats the following shape: (610, 1032)
1079
(1079, 516)
(1079, 516) (1079,)
(1074, 516)
(1064, 516)
Appending to feats the following shape: (532, 1032)
1445
(1445, 516)
(1445, 516) (1445,)
(1433, 516)
(1423, 516)
Appending to feats the following shape: (711, 1032)
1074
(1074, 516)
(1074, 516) (1074,)
(1064, 516)
(1054, 516)
Appending to feats the following shape: (527, 1032)
1368
(1368, 516)
(1368, 516) (1368,)
(1182, 516)
(1172, 516)
Appending to feats the following shape: (586, 1032)
1895
(1895, 516)
(1895, 516) (1895,)
(45, 516)
(35, 516)
Appending to feats the following shape: (17, 1032)
1997
(1997, 516)
(1997, 516) (1997,)
(1923, 516)
(1913, 516)
Appending to feats the following shape: (956, 1032)
1201
(1201, 516)
(1201, 516) (1201,)
(887, 516)
(877, 516)
Appending to feats the following shape: (438, 1032)
1899
(1899, 516)
(1899, 516) (1899,)
(1846, 516)
(1836, 516)
Appending to feats the

In [60]:
def gen_data_matrix(data_map):
    X = None
    y = None

    idx2label = {}
    for idx, (label, data) in enumerate(data_map.items()):
        if len(data) == 0:
            continue
            
        idx2label[idx] = label
        print(idx, label)
            
        if X is None:
            X = data
            y = np.ones(X.shape[0]) * idx
        else:
            X = np.vstack((X, data))
            y = np.hstack((y, np.ones(data.shape[0]) * idx))

#     print(X.shape)
#     print(y.shape)
    return X, y, idx2label
    
X_train, y_train, idx2label = gen_data_matrix(data_map_train)
X_test, y_test, _ = gen_data_matrix(data_map_test)

0 relaxed
1 tense
0 relaxed
1 tense


In [61]:
# print(", ".join([str(d) for d in X_train[0]]))

# all_ts_run = np.array(all_ts_run)
# deltas = all_ts_run[1:] - all_ts_run[:-1]
# print(deltas.mean())
# print(np.median(deltas))
# print(deltas.std())

In [62]:
# X_train = np.vstack((all_fft_base, all_fft_run))
# y_train = np.hstack((np.zeros(all_fft_base.shape[0]), np.ones(all_fft_run.shape[0])))
# X_train = np.vstack((all_fft_heli, all_fft_run, all_fft_base))
# y_train = np.hstack((np.zeros(all_fft_heli.shape[0]), np.ones(all_fft_run.shape[0]), 2 * np.ones(all_fft_base.shape[0])))

# print(all_fft_base.shape, all_fft_run.shape, all_fft_heli.shape)
print(X_train.shape, y_train.shape)
print(X_train.dtype, y_train.dtype)

(5295, 1032) (5295,)
float64 float64


In [63]:
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=lambda x: x[1], reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("{2}: Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores),
              i + 1))
        print("Parameters: {0}".format(score.parameters))

In [82]:
from sklearn.svm import SVC
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.neighbors import *
from sklearn.pipeline import *
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import *

# clf = DummyClassifier()
# clf = RandomForestClassifier()
# clf = AdaBoostClassifier()
# clf = SVC()
# clf = SGDClassifier()
clf = LogisticRegression()

Cs = [0.01, 0.1, 1.0, 2.0, 10, 25]
# tuned_parameters = [{'kernel': ['rbf'], 'gamma': [0.1, 0.01, 0.001, 0.0001],
#                      'C': [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 25, 50]},
#                     {'kernel': ['linear'], 'C': [0.001, 0.01, 0.05, 0.1, 0.5, 10, 50, 100]}]
# cws = ['balanced', {0: 3, 1: 1}, {0: 1, 1: 1}]
tuned_parameters = [ # SVC
    {
        'clf__kernel': ['linear'],
        'clf__C': Cs,
    },
#     {
#         'clf__kernel': ['rbf'],
#         'clf__gamma': [0.1, 0.01, 0.001], #, 0.0001],
#         'clf__C': Cs,
#         'epsilon': []
#     },
#     {
# #         'kernel': ['poly']
#     }
]
tuned_parameters = {
#     'clf__n_estimators': [2, 3, 5, 10, 15], #, 15, 20], #, 25, 50], #, 75, 100, 125]#, 150],
      'clf__n_estimators': [10, 50, 100],
#     'clf__class_weight': ['balanced', None],
#     'scaler__with_mean': [True, False],
#     'scaler__with_std': [True, False],
}

tuned_parameters = { # LogReg
    # C = Inverse of regularization strength; must be a positive float. 
    # Like in support vector machines, smaller values specify stronger regularization.
#     'clf__C': [0.01, 0.1, 0.25, 1.0],
    'clf__C': [0.0005, 0.001, 0.005, 0.01, 0.05]#, 0.1, 0.25]#, 1.0, 5.0, 10.0]
#     'scaler__with_mean': [True, False],
#     'scaler__with_std': [True, False],
}

# tuned_parameters = {  # SGDClassifier
# #     'clf__epsilon' : [0.01, 0.1, 1.0],
# #     'clf__alpha': [0.05, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
#     'clf__alpha': [2.5, 5.0, 10.0, 15.0],
#     'clf__class_weight': ['balanced', None],
# }

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', clf)
])

print(np.histogram(y_train)[0])
grid = GridSearchCV(pipeline, tuned_parameters, cv=10, scoring='f1_micro', n_jobs=-2, verbose=1)

print(X_train.shape)
print("Kicking off grid search...")

# Hack to suppress tons of output when doing feature selection (doesn't impact the actual feature selection)
# np.seterr(all='ignore')
res = grid.fit(X_train, y_train)

[2380    0    0    0    0    0    0    0    0 2915]
(5295, 1032)
Kicking off grid search...
Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-2)]: Done  50 out of  50 | elapsed:   28.5s finished


In [83]:
report(res.grid_scores_, n_top=5)

1: Mean validation score: 0.774 (std: 0.044)
Parameters: {'clf__C': 0.001}
2: Mean validation score: 0.774 (std: 0.040)
Parameters: {'clf__C': 0.005}
3: Mean validation score: 0.772 (std: 0.049)
Parameters: {'clf__C': 0.0005}
4: Mean validation score: 0.768 (std: 0.042)
Parameters: {'clf__C': 0.01}
5: Mean validation score: 0.764 (std: 0.046)
Parameters: {'clf__C': 0.05}


/Users/andrei/anaconda3/envs/oxh/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [84]:
y_pred_train = grid.predict(X_train)
print(idx2label)
confusion_matrix(y_train, y_pred_train)

{0: 'relaxed', 1: 'tense'}


array([[2272,  108],
       [ 270, 2645]])

In [85]:
yy=res.predict(X_test)
# print(np.bincount(y_test.astype(np.int64)))
# print(yy)
print(y_test[0:10], y_test[-10:])
counts1 = np.bincount(yy[:469].astype(np.int64))
counts2 = np.bincount(yy[469:].astype(np.int64))
print(counts1, counts2)
print(np.argmax(counts1), np.argmax(counts2))
print(res.best_estimator_)
# print(f1_score(y_test, yy, average='macro'))


[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[279 190] [338 505]
0 1
Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


In [ ]:
# LogReg: [446 466 250]
# LinSVC: [614 393 155] -> overfits
# SGDClassifier: [614 393 155] -> overfits (same as LinSVC; just faster)
# RFC: [481 340 341], [597 372 193] 
# RFC, different case: [738 744 194], [676 716 284] (middle one is correct)

# stacking every 2 consecutive feature vectors: 
#  - [383 412  48] with LogReg (best, so far)
#  - [354 338 151] with RFC
#  - [456 372  15] with SVC
#  - [208 594  41] with AdaBoost (100 estimators)

# Added way more data (e.g. the 2 duolingo sessions)
#  - [  0 199 270] [ 42 108 693] == [2, 2] with AdaBoost (not ignoring baseline; under-represented)
#  - <bad> with SGDClassifier (no more baseline class)
#  - [258 211] [273 570] == [0, 1] with LogReg
#  - [257 212] [279 564] == [0, 1] with LogReg, more regularization
#  - [279 190] [338 505] == [0, 1] with LogReg, even more regularization

In [ ]:
# print(res)

# Saving the model for later use

In [86]:
import pickle

with open('../model-lr.pkl', 'wb') as f:
    pickle.dump(res.best_estimator_, f)
    print("Successfully dumped model pickle.")

Successfully dumped model pickle.
